# 🛍️ Mall Customers – Cluster Analysis (K-Means + Hierárquico)
---
Notebook completo com análise de cluster usando **K-Means**, **Dendrograma**, **Elbow Method** e **ANOVA**.

# 🗂️ Fonte da Base de Dados
A base utilizada é o Mall Customers Dataset, disponível publicamente no Kaggle:
🔗 https://www.kaggle.com/datasets/shwetabh123/mall-customers

## 📦 0. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans
from scipy.stats import zscore
import pingouin as pg


## 📂 1. Leitura da Base & EDA Inicial

In [ ]:
mall = pd.read_csv('Mall_Customers.csv')
print(mall.info())

# 1.1 Média de Renda Anual por Gênero
mall.groupby('Genre')['Annual Income (k$)'].mean()

## 🎛️ 2. Padronização das Variáveis Numéricas

In [ ]:
mallnumeric = mall.drop(columns=['CustomerID', 'Genre'])
mallpad = mallnumeric.apply(zscore, ddof=1)
testeanova = mallpad.copy() # Duplicando para ANOVA futura

## 🌳 3. Cluster Hierárquico – Dendrograma

In [ ]:
mall_compl = sch.linkage(mallpad, metric='euclidean', method='complete')

# maior salto
dists = mall_compl[:, 2]
saltos = np.diff(dists)
i = np.argmax(saltos)
print(f"O maior salto está entre: {dists[i]:.2} e {dists[i+1]:.2}")

#⚠️ Vamos considerar 4 clusters, considerando as evidências anteriores!
# plot
plt.figure(figsize=(12, 6))
sch.dendrogram(mall_compl, color_threshold=4, labels=list(mall['CustomerID']))
plt.axhline(y=4, color='red', linestyle='--')
plt.title("Dendrograma – Complete Linkage")
plt.show()

## 📉 4. Escolha do Número Ideal de Clusters – Elbow Method

In [ ]:
wcss = []
ks = range(1, 11)

for k in ks:
    kmeans = KMeans(n_clusters=k, init='random', random_state=42)
    kmeans.fit(mallpad)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(12, 6))
plt.plot(ks, wcss, marker='o')
plt.xlabel("Número de Clusters (k)")
plt.ylabel("WCSS")
plt.title("Método do Cotovelo (Elbow Method)")
plt.show()

## 🎯 5. Modelo Final – KMeans com k = 4

In [ ]:
cluster_final = KMeans(n_clusters=4, init='random', random_state=42).fit(mallpad)
cluster_KMeans = cluster_final.labels_

mall['Cluster_KMeans'] = cluster_KMeans
testeanova['Cluster_KMeans'] = cluster_KMeans
mall.head()

## 📊 6. ANOVA – Comparação Estatística Entre os Clusters

In [ ]:
print("\nANOVA – Age")
print(pg.anova(dv='Age', between='Cluster_KMeans', data=testeanova, detailed=True).T)

print("\nANOVA – Annual Income (k$)")
print(pg.anova(dv='Annual Income (k$)', between='Cluster_KMeans', data=testeanova, detailed=True).T)

print("\nANOVA – Spending Score (1-100)")
print(pg.anova(dv='Spending Score (1-100)', between='Cluster_KMeans', data=testeanova, detailed=True).T)